In [25]:
"""
Implementação prática de KNN usando a base Iris (UCI), com:
- Leitura do arquivo iris.data
- Separação treino/teste
- Padronização (StandardScaler)
- KNN para K em {1, 3, 5, 7}
- Impressão de acurácia e relatório de classificação

Este script roda direto: basta ter o iris.data na mesma pasta.
"""

import pathlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


In [26]:
# ===== Configurações gerais =====
ARQUIVO_IRIS = "iris.data"          # deve estar na mesma pasta deste script
VALORES_K = [1, 3, 5, 7]            # Ks a serem testados
TEST_SIZE = 0.3                     # 30% para teste
RANDOM_STATE = 42                   # semente para reprodutibilidade

In [27]:
def carregar_iris(caminho_arquivo: pathlib.Path) -> tuple[np.ndarray, np.ndarray]:
    """
    Lê o arquivo iris.data e retorna:
    - X: matriz (n_amostras, 4) com atributos numéricos
    - y: vetor (n_amostras,) com as classes (strings)
    """
    if not caminho_arquivo.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho_arquivo}")

    # iris original não tem cabeçalho; são 5 colunas: 4 numéricas + 1 classe
    df = pd.read_csv(
        caminho_arquivo,
        header=None,
        names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
    )

    # remove linhas completamente vazias (às vezes há uma linha em branco no final)
    df = df.dropna(how="any")

    X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]].to_numpy()
    y = df["class"].to_numpy()

    return X, y

In [28]:
def rodar_knn_para_k(X: np.ndarray, y: np.ndarray, k: int) -> None:
    """
    Divide em treino/teste, monta um Pipeline (padronização + KNN),
    treina o modelo e imprime métricas para o valor de K informado.
    """
    # separa treino/teste estratificado (mantém proporção de classes)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
        stratify=y
    )

    # Pipeline: primeiro padroniza, depois aplica KNN
    modelo = Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(
            n_neighbors=k,
            metric="minkowski",    # padrão
            p=2                    # p=2 => distância Euclidiana
        ))
    ])

    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    acc = accuracy_score(y_test, y_pred)

    print("=" * 60)
    print(f"KNN com K = {k}")
    print(f"Acurácia no conjunto de teste: {acc:.4f}")
    print("\nRelatório de classificação:")
    print(classification_report(y_test, y_pred, digits=4))



In [ ]:
def main():
    caminho_iris = pathlib.Path(ARQUIVO_IRIS)
    print("Carregando base Iris de:", caminho_iris)
    X, y = carregar_iris(caminho_iris)
    print(f"Total de instâncias: {X.shape[0]}")
    print(f"Número de atributos: {X.shape[1]}")
    print("Classes presentes:", sorted(set(y)))
    print()

    for k in VALORES_K:
        rodar_knn_para_k(X, y, k)

    print("=" * 60)
    print("Execução concluída com sucesso.")

main()

Carregando base Iris de: iris.data
Total de instâncias: 150
Número de atributos: 4
Classes presentes: ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

KNN com K = 1
Acurácia no conjunto de teste: 0.9333

Relatório de classificação:
                 precision    recall  f1-score   support

    Iris-setosa     1.0000    1.0000    1.0000        15
Iris-versicolor     0.8333    1.0000    0.9091        15
 Iris-virginica     1.0000    0.8000    0.8889        15

       accuracy                         0.9333        45
      macro avg     0.9444    0.9333    0.9327        45
   weighted avg     0.9444    0.9333    0.9327        45

KNN com K = 3
Acurácia no conjunto de teste: 0.9111

Relatório de classificação:
                 precision    recall  f1-score   support

    Iris-setosa     1.0000    1.0000    1.0000        15
Iris-versicolor     0.7895    1.0000    0.8824        15
 Iris-virginica     1.0000    0.7333    0.8462        15

       accuracy                         0.9111    